# PyMongo

In [1]:
from pymongo import MongoClient, ASCENDING, DESCENDING
from datetime import datetime

server = "mongodb://172.17.0.1:27017"
# server = "mongodb://15.15.165.35:27017"
client = MongoClient(server)
db = client.test

## Insert Data with PyMongo

In [2]:
result = db.abc.insert_one(
    {
        "address": {
            "street": "2 Avenue",
            "zipcode": "10075",
            "building": "1480",
            "coord": [-73.9557413, 40.7720266]
        },
        "borough": "Manhattan",
        "cuisine": "Italian",
        "grades": [
            {
                "date": datetime.strptime("2014-10-01", "%Y-%m-%d"),
                "grade": "A",
                "score": 11
            },
            {
                "date": datetime.strptime("2014-01-16", "%Y-%m-%d"),
                "grade": "B",
                "score": 17
            }
        ],
        "name": "Vella",
        "restaurant_id": "41704620"
    }
)
print(result.inserted_id)

5a0bcb110c167805014a46ec


In [28]:
## Find or Query Data with PyMongo 

In [32]:
cursor = db.restaurants.find({"borough": "Manhattan", 
                              "address.zipcode": "10075", 
                              "grades.grade": "B", 
                              "grades.score": {"$lt": 10} ,
                              "grades.score": {"$gt": 30},   #看上去，如果对于同一个字段的条件，最后一个才生效
                             }).sort([("borough", ASCENDING),
                                      ("address.zipcode", ASCENDING)
                                     ])
i = 0
for document in cursor: 
    i = i + 1
    print(document)
print(i)    


{'_id': ObjectId('5a0ace5d37cd621475e75b23'), 'address': {'building': '1011', 'coord': [-73.9623333, 40.7757194], 'street': 'Madison Avenue', 'zipcode': '10075'}, 'borough': 'Manhattan', 'cuisine': 'American', 'grades': [{'date': datetime.datetime(2014, 5, 2, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2013, 10, 21, 0, 0), 'grade': 'B', 'score': 15}, {'date': datetime.datetime(2013, 4, 19, 0, 0), 'grade': 'B', 'score': 0}, {'date': datetime.datetime(2012, 11, 21, 0, 0), 'grade': 'C', 'score': 40}, {'date': datetime.datetime(2012, 4, 9, 0, 0), 'grade': 'B', 'score': 17}], 'name': 'Viand Cafe', 'restaurant_id': '40369753'}
{'_id': ObjectId('5a0ace5d37cd621475e7670c'), 'address': {'building': '1526', 'coord': [-73.95202990000001, 40.772737], 'street': '1 Avenue', 'zipcode': '10075'}, 'borough': 'Manhattan', 'cuisine': 'Pizza/Italian', 'grades': [{'date': datetime.datetime(2014, 11, 19, 0, 0), 'grade': 'Z', 'score': 50}, {'date': datetime.datetime(2014, 6, 6, 0, 0), 'grad

In [73]:
# Logical OR
cursor = db.restaurants.find({"$or": [{"cuisine": "Italian"}, {"address.zipcode": "10075"}], 
                              "borough": "Manhattan",
                              "grades.grade": "B", 
                              "grades.score": {"$gt": 60}
                             })

i = 0
for document in cursor: 
    i = i + 1
    print(document)
print(i)    

{'_id': ObjectId('5a0ba6d5532ce2a8549ccabc'), 'address': {'building': '1', 'coord': [-74.00939799999999, 40.715763], 'street': 'Hudson Street', 'zipcode': '10013'}, 'borough': 'Manhattan', 'cuisine': 'Italian', 'grades': [{'date': datetime.datetime(2014, 3, 10, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2013, 9, 26, 0, 0), 'grade': 'B', 'score': 19}, {'date': datetime.datetime(2013, 3, 19, 0, 0), 'grade': 'C', 'score': 64}, {'date': datetime.datetime(2012, 5, 8, 0, 0), 'grade': 'A', 'score': 12}, {'date': datetime.datetime(2011, 12, 13, 0, 0), 'grade': 'B', 'score': 18}, {'date': datetime.datetime(2011, 7, 5, 0, 0), 'grade': 'A', 'score': 5}], 'name': 'Acappella Restaurant', 'restaurant_id': '40399334'}
{'_id': ObjectId('5a0ba6d5532ce2a8549cd131'), 'address': {'building': '231', 'coord': [-73.9772294, 40.7527262], 'street': 'Grand Central Station', 'zipcode': '10017'}, 'borough': 'Manhattan', 'cuisine': 'Italian', 'grades': [{'date': datetime.datetime(2015, 1, 7, 0, 0

## Update Data with PyMongo

In [52]:
result = db.restaurants.update_one(
    {"name": "Juni"},
    {
        "$set": {"cuisine": "American (New)"},
        "$currentDate": {"lastModified": True}  #为何没有发现哪里有这个字段。
    }
)

print(result.matched_count)
print(result.modified_count)

1
1


In [44]:
result = db.restaurants.update_one(
    {"restaurant_id": "41156888"},
    {"$set": {"address.street": "East 32st Street"}}
)
print(result.matched_count)
print(result.modified_count)

1
0


In [79]:
result = db.restaurants.update_many(
    {"address.zipcode": "10016"},
    {
        "$set": {"cuisine": "Category To Be Determined"},
        "$currentDate": {"lastModified": True}
    }
)
print(result.matched_count)
print(result.modified_count)

433
433


Replace将会完全替换以前的文档，这样以前的数据就完全没有了

In [77]:
result = db.restaurants.replace_one(
    {"restaurant_id": "41704620"},
    {
        "name": "Vella 2",
        "address": {
            "coord": [-73.9557413, 40.7720266],
            "building": "1480",
            "street": "2 Avenue",
            "zipcode": "10075"
        }
    }
)

print(result.matched_count)
print(result.modified_count)

1
1


## Remove Data with PyMongo



In [63]:
result = db.restaurants.delete_many({"borough": "Manhattan", "grades.score": {"$gt": 30}})
print(result.deleted_count)

## Remove All Documents
# result = db.restaurants.delete_many({})

831


### Drop a Collection

In [ ]:
#db.restaurants.drop()

## Data Aggregation with PyMongo

In [64]:
cursor = db.restaurants.aggregate(
    [
        {"$group": {"_id": "$borough", "count": {"$sum": 1}}}
    ]
)

i = 0
for document in cursor: 
    i = i + 1
    print(document)
print(i)    


{'_id': None, 'count': 2}
{'_id': 'Missing', 'count': 51}
{'_id': 'Staten Island', 'count': 969}
{'_id': 'Brooklyn', 'count': 6086}
{'_id': 'Queens', 'count': 5656}
{'_id': 'Bronx', 'count': 2338}
{'_id': 'Manhattan', 'count': 9433}
7


In [65]:
cursor = db.restaurants.aggregate(
    [
        {"$match": {"borough": "Queens", "cuisine": "Brazilian"}},
        {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}}
    ]
)

i = 0
for document in cursor: 
    i = i + 1
    print(document)
print(i)  

{'_id': '11377', 'count': 1}
{'_id': '11103', 'count': 1}
{'_id': '11106', 'count': 3}
{'_id': '11368', 'count': 1}
{'_id': '11101', 'count': 2}
5


## Indexes with PyMongo

In [70]:
db.restaurants.create_index([("cuisine", ASCENDING)])
db.restaurants.create_index([
    ("cuisine", ASCENDING),
    ("address.zipcode", DESCENDING)
])

'cuisine_1_address.zipcode_-1'

# mongo version
## import dataset by mongoimport

wget https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json
mongoimport --db test --collection restaurants --drop --file ./primer-dataset.json

mongo
use test

## Insert Data with the mongo Shell


db.restaurants.insert(
   {
      "address" : {
         "street" : "2 Avenue",
         "zipcode" : "10075",
         "building" : "1480",
         "coord" : [ -73.9557413, 40.7720266 ]
      },
      "borough" : "Manhattan",
      "cuisine" : "Italian",
      "grades" : [
         {
            "date" : ISODate("2014-10-01T00:00:00Z"),
            "grade" : "A",
            "score" : 11
         },
         {
            "date" : ISODate("2014-01-16T00:00:00Z"),
            "grade" : "B",
            "score" : 17
         }
      ],
      "name" : "Vella",
      "restaurant_id" : "41704620"
   }
)

## Find or Query Data with the mongo Shell

In [4]:
db.restaurants.find({"borough": "Manhattan", 
              "address.zipcode": "10075", 
              "grades.grade": "B", 
              "grades.score": {$lt: 10} ,
              "grades.score": {$gt: 30},   
             }).sort({"borough": 1, "address.zipcode": 1})



SyntaxError: invalid syntax (<ipython-input-4-f1b5f6347d1f>, line 4)

In [3]:
 db.restaurants.find({"$or": [{"cuisine": "Italian"}, {"address.zipcode": "10075"}], 
                              "borough": "Manhattan",
                              "grades.grade": "B", 
                              "grades.score": {"$gt": 60}
                             })

## Update Data with the mongo Shell

db.restaurants.update(
    {"address.zipcode": "10016"},
    {
        "$set": {"cuisine": "Category To Be Determined"},
        "$currentDate": {"lastModified": true}
    }
)

db.restaurants.update(
    {"address.zipcode": "10016"},
    {
        "$set": {"cuisine": "Category To Be Determined"},
        "$currentDate": {"lastModified": true}
    },
    { multi: true}
)

result = db.restaurants.update(
    {"restaurant_id": "41704620"},
    {
        "name": "Vella 2",
        "address": {
            "coord": [-73.9557413, 40.7720266],
            "building": "1480",
            "street": "2 Avenue",
            "zipcode": "10075"
        }
    }
)

## Remove Data with the mongo Shell

db.restaurants.remove({"borough": "Manhattan", "grades.score": {"$gt": 30}})

#db.restaurants.remove( { } )
#db.restaurants.drop()

## Data Aggregation with the mongo Shell

db.restaurants.aggregate(
    [
        {"$group": {"_id": "$borough", "count": {"$sum": 1}}}
    ]
)

db.restaurants.aggregate(
    [
        {"$match": {"borough": "Queens", "cuisine": "Brazilian"}},
        {"$group": {"_id": "$address.zipcode", "count": {"$sum": 1}}}
    ]
)

## Indexes with the mongo Shell

db.restaurants.createIndex( { "cuisine": 1 } )
db.restaurants.createIndex( { "cuisine": 1, "address.zipcode": -1 } )

## Drivers

https://docs.mongodb.com/getting-started/shell/drivers/
https://docs.mongodb.com/manual/